# Chargement de données
## Auteur Linda MARTIN

La construction des données à charger dans le model sont à catégoriser. Un modèle qui prend en entrée 558 colonnes et 424 en sortie, sera trop couteux en mémoire et en temps d'apprentissage. Pour rappel, le jeu de donnée de test doit être réserver que pour la partie final de prédiction.


In [1]:
# import libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [2]:
# load data
train_df = pd.read_csv('../data/train.csv', index_col= 'date_id')
test_df = pd.read_csv('../data/test.csv', index_col = 'date_id')

train_labels_df = pd.read_csv('../data/train_labels.csv', index_col = 'date_id')
target_pairs_df = pd.read_csv('../data/target_pairs.csv')



In [3]:
display(train_df.tail())
display(test_df.tail())
display(train_df.shape, test_df.shape)

,LME_AH_Close,LME_CA_Close,LME_PB_Close,LME_ZS_Close,JPX_Gold_Mini_Futures_Open,JPX_Gold_Rolling-Spot_Futures_Open,JPX_Gold_Standard_Futures_Open,JPX_Platinum_Mini_Futures_Open,JPX_Platinum_Standard_Futures_Open,JPX_RSS3_Rubber_Futures_Open,...,FX_GBPCAD,FX_CADCHF,FX_NZDCAD,FX_NZDCHF,FX_ZAREUR,FX_NOKGBP,FX_NOKCHF,FX_ZARCHF,FX_NOKJPY,FX_ZARGBP
date_id,,,,,,,,,,,,,,,,,,,,,
1912,2450.0,9523.5,1961.5,2676.5,15086.0,15440.0,15085.0,4461.5,4467.0,311.7,...,1.864661,0.598318,0.827529,0.495125,0.049224,0.072574,0.080968,0.046175,14.058107,0.041388
1913,2471.5,9519.5,1980.5,2710.5,15165.0,15509.0,15162.0,4495.0,4490.0,324.2,...,1.863539,0.594400,0.824390,0.490018,0.049409,0.072828,0.080671,0.046113,14.082236,0.041630
1914,2471.5,9533.5,1974.0,2693.0,15040.0,15477.0,15044.0,4544.5,4555.0,322.6,...,1.860067,0.595250,0.822392,0.489529,0.049095,0.073232,0.081083,0.045901,14.126606,0.041457
1915,2456.0,9500.5,1970.0,2697.5,15420.0,15752.0,15420.0,4670.0,4685.0,320.2,...,1.859624,0.597780,0.817224,0.488520,0.049205,0.073018,0.081170,0.045987,14.095322,0.041368
1916,2463.5,9610.0,1991.0,2701.5,15416.0,15802.0,15415.0,4730.5,4751.0,322.0,...,1.860472,0.597644,0.822219,0.491394,0.049321,0.073058,0.081233,0.046037,14.107311,0.041404


,LME_AH_Close,LME_CA_Close,LME_PB_Close,LME_ZS_Close,JPX_Gold_Mini_Futures_Open,JPX_Gold_Rolling-Spot_Futures_Open,JPX_Gold_Standard_Futures_Open,JPX_Platinum_Mini_Futures_Open,JPX_Platinum_Standard_Futures_Open,JPX_RSS3_Rubber_Futures_Open,...,FX_CADCHF,FX_NZDCAD,FX_NZDCHF,FX_ZAREUR,FX_NOKGBP,FX_NOKCHF,FX_ZARCHF,FX_NOKJPY,FX_ZARGBP,is_scored
date_id,,,,,,,,,,,,,,,,,,,,,
1912,2450.0,9523.5,1961.5,2676.5,15086.0,15440.0,15085.0,4461.5,4467.0,311.7,...,0.598318,0.827529,0.495125,0.049224,0.072574,0.080968,0.046175,14.058107,0.041388,True
1913,2471.5,9519.5,1980.5,2710.5,15165.0,15509.0,15162.0,4495.0,4490.0,324.2,...,0.594400,0.824390,0.490018,0.049409,0.072828,0.080671,0.046113,14.082236,0.041630,True
1914,2471.5,9533.5,1974.0,2693.0,15040.0,15477.0,15044.0,4544.5,4555.0,322.6,...,0.595250,0.822392,0.489529,0.049095,0.073232,0.081083,0.045901,14.126606,0.041457,True
1915,2456.0,9500.5,1970.0,2697.5,15420.0,15752.0,15420.0,4670.0,4685.0,320.2,...,0.597780,0.817224,0.488520,0.049205,0.073018,0.081170,0.045987,14.095322,0.041368,True
1916,2463.5,9610.0,1991.0,2701.5,15416.0,15802.0,15415.0,4730.5,4751.0,322.0,...,0.597644,0.822219,0.491394,0.049321,0.073058,0.081233,0.046037,14.107311,0.041404,True


(1917, 557)

(90, 558)

Nous avons les 90 dernières dates du fichier d'entrainement qui correspond aux données de tests.

In [5]:
# on supprime les id_data du fichier test pour le jeu de donnée
cutoff_date = train_df.index[-90]
real_train_df = train_df[train_df.index < cutoff_date]
display(real_train_df.tail())

,LME_AH_Close,LME_CA_Close,LME_PB_Close,LME_ZS_Close,JPX_Gold_Mini_Futures_Open,JPX_Gold_Rolling-Spot_Futures_Open,JPX_Gold_Standard_Futures_Open,JPX_Platinum_Mini_Futures_Open,JPX_Platinum_Standard_Futures_Open,JPX_RSS3_Rubber_Futures_Open,...,FX_GBPCAD,FX_CADCHF,FX_NZDCAD,FX_NZDCHF,FX_ZAREUR,FX_NOKGBP,FX_NOKCHF,FX_ZARCHF,FX_NOKJPY,FX_ZARGBP
date_id,,,,,,,,,,,,,,,,,,,,,
1822,2571.5,9091.5,1975.0,2868.0,13620.5,13895.0,13624.0,4802.0,4798.0,363.3,...,1.772281,0.630820,0.806869,0.508989,0.051018,0.071398,0.079822,0.047927,13.745669,0.042869
1823,2578.5,9094.5,1958.5,2865.5,NaN,NaN,NaN,NaN,NaN,NaN,...,1.756813,0.637624,0.804638,0.513056,0.051247,0.071740,0.080362,0.048176,13.807214,0.043007
1824,2560.0,9154.0,1966.0,2862.5,13701.5,13960.0,13701.0,4790.5,4796.0,368.6,...,1.752560,0.635810,0.804070,0.511236,0.051273,0.071971,0.080197,0.048221,13.890139,0.043275
1825,2601.5,9167.5,1935.0,2863.0,13583.5,13873.0,13587.0,4740.0,4757.0,376.6,...,1.753901,0.636714,0.806343,0.513410,0.051739,0.072166,0.080590,0.048587,13.803503,0.043508
1826,2636.5,9230.5,1970.5,2874.5,13635.0,13918.0,13621.0,4690.0,4683.0,383.7,...,1.761285,0.632962,0.807244,0.510955,0.051597,0.071946,0.080207,0.048433,13.677595,0.043445


## Restructuration par caractéristiques
On restructure de la manière à avoir les colonnes par catégories de produit :
['date', 'id', 'close', 'open', 'high', 'low', 'volume', 'sprice', 'interest']

In [6]:

warnings.filterwarnings("ignore")
# Future block-specific warnings
warnings.filterwarnings(
    'ignore',
    category=FutureWarning,
    message="The behavior of DataFrame concatenation with empty or all-NA entries is deprecated."
)
warnings.filterwarnings("ignore", category=RuntimeWarning, message="invalid value encountered in ")

def get_preprocess_data(df, cond):
    # Fonction pour obtenir les données prétraitées en fonction de la condition
    if cond.Column.size > 0:
        return df[cond.Column.values[0]]
    else:
        return None
    
def get_train_info(df):
    """ Construction d'une data des entêtes de colonnes.
    Args:
        df (pd.DataFrame): Input dataframe d'entrainement.
    Returns:
        pd.DataFrame: Détail des informations de chaque colonne.
    """
    df_names = df.columns
    # Fonction pour nettoyer et split les noms
    def clean_and_split(name):
        name = name.replace("open_interest", "open interest")
        name = name.replace("settlement_price", "settlement price")
        name = name.replace("US_Stock", "US Stock")
        name = name.replace("adj_close", "Close")
        name = name.replace("adj_", "adjusted ")
        name = name.replace("-", "_")
        return name.split("_")

    # Création du DataFrame d'infos
    df_info = pd.DataFrame(
        {
        'Column': df_names,
        'Split': [clean_and_split(name) for name in df_names]
    })

    df_info['Category'] = df_info['Split'].apply(lambda x: x[0])    
    df_info['Ticker'] = df_info['Split'].apply(
    lambda x: "_".join(x[1:-1]) if len(x) > 2 else x[-1] if len(x) == 2 else ""
    )
    df_info['Type'] = df_info['Split'].apply(lambda x: x[-1])

    # Nettoyage final
    df_info['Ticker'] = df_info.apply(
        lambda row: row['Type'] if row['Ticker'] == "" else row['Ticker'], axis=1
    )
    df_info['Column_Id'] = df_info.index + 1

    # Sélection des colonnes finales
    df_info = df_info[['Column_Id', 'Column', 'Category', 'Ticker', 'Type']]    
    return df_info

def preprocess(df):
    """ Pré-process pour regrouper les données par catégorie.
     Args:
        df (pd.DataFrame): Input dataframe d'entrainement.
        Returns: pd.DataFrame par catégorie."""
    
    df_processed = df.copy()
    df_info = get_train_info(df_processed)

    df_processed.reset_index(inplace=True)
    # On renomme la date_id en date
    df_processed = df_processed.rename({'date_id': 'date'}, axis='columns')
    # Initialisation du DataFrame résultat
    result = pd.DataFrame(columns=['date', 'id', 'close', 'open', 'high', 'low', 'volume', 'sprice', 'interest'])
    
    for  Category  in df_info.groupby('Category').groups.keys():
         for label in df_info[(df_info.Category==Category)].groupby('Ticker').groups.keys():
            temp_df = pd.DataFrame()
            temp_df['date'] = df_processed['date']
            temp_df['id'] = f'{Category}_{label}'

            if Category in ['FX','LME']:
                temp_df['close'] = df_processed[df_info[(df_info.Category==Category) & (df_info.Ticker==label)].Column.values[0]]
                temp_df['open'] = None
                temp_df['high'] = None
                temp_df['low'] = None
                temp_df['volume'] = None
                temp_df['sprice'] = None
                temp_df['interest'] = None
            else:
                temp_df['close'] = get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['Close', 'adjusted close']))])
                temp_df['open'] = get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['Open','adjusted open']))])
                temp_df['high'] = get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['High','adjusted high']))])
                temp_df['low'] = get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['Low','adjusted low']))])
                temp_df['volume'] = get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['Volume', 'adjusted volume']))])
                temp_df['sprice'] = get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['settlement price','adjusted settlement price']))])
                temp_df['interest'] = get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['open interest','adjusted open interest']))])
            result = pd.concat([result, temp_df], ignore_index=True)
      
    # Réinitialiser l'index
    result = result.reset_index(drop=True)
    
    # Trier par date et id 
    result = result.sort_values(['date', 'id']).reset_index(drop=True)
    
    return result
        


In [7]:
df_preprocess = preprocess(real_train_df)
display(df_preprocess.head())

,date,id,close,open,high,low,volume,sprice,interest
0,0,FX_AUDCAD,0.979601,NaN,NaN,NaN,NaN,NaN,NaN
1,0,FX_AUDCHF,0.761027,NaN,NaN,NaN,NaN,NaN,NaN
2,0,FX_AUDJPY,87.933498,NaN,NaN,NaN,NaN,NaN,NaN
3,0,FX_AUDNZD,1.103011,NaN,NaN,NaN,NaN,NaN,NaN
4,0,FX_AUDUSD,0.783393,NaN,NaN,NaN,NaN,NaN,NaN


A cette structure on ajoute des fonctions comme le déplacement de données pour le traitement de lag et des moyennes dans différentes zones de temps sur chaque colonne.  

In [8]:
from typing import List

class FeatureEngineer:
    """Creation des fonctions """
    def __init__(self):
        pass

    def add_lag_features(self,
        df: pd.DataFrame, 
        lags: List[int], 
        date_col: str = 'date'
        ) -> pd.DataFrame:
        """
        Add lag features for specified columns and lags.
        """
        df = df.sort_values(date_col)
        cols = set(df.columns)
        cols.remove('id')
        cols.remove('date')
        for col in cols:
            for lag in lags:
                df[f'{col}_lag{lag}'] = df.groupby('id')[col].shift(lag)
        return df
    
    def add_rolling_features(self,
        df: pd.DataFrame,
        windows: List[int],
        date_col: str = 'date') -> pd.DataFrame:
        """ 
        Add rolling mean and std features for specified columns and windows.
        """
        df = df.sort_values(date_col)
        cols = set(df.columns)
        cols.remove('id')
        cols.remove('date')
        for col in cols:
            for window in windows:
                df[f'{col}_rollmean{window}'] = df.groupby('id')[col].transform(lambda x: x.rolling(window, min_periods=1).mean())
                df[f'{col}_rollstd{window}'] = df.groupby('id')[col].transform(lambda x: x.rolling(window, min_periods=1).std())
                df[f'{col}_rollmin{window}'] = df.groupby('id')[col].transform(lambda x: x.rolling(window, min_periods=1).min())
                df[f'{col}_rollmax{window}'] = df.groupby('id')[col].transform(lambda x: x.rolling(window, min_periods=1).max())
        return df
    
    def prepare_features(self, df: pd.DataFrame) ->pd.DataFrame:
        """Engineer features for training and testing data"""
        try:
            # add lag
            df_result = df.copy()
            df_result = self.add_lag_features(df_result, lags=[1, 2, 3, 5, 7])  
            # Add rolling features
            df_result = self.add_rolling_features(df_result, windows=[5, 10, 15])
            # Handle missing values
            df_result = df_result.fillna(method='ffill').fillna(method='bfill').fillna(0)

            return df_result
        except Exception as e:
            print(f"Feature preparation failed: {e}")
            raise

In [ ]:
class FeatureTarget:
    """Class to handle target feature engineering"""
    def __init__(self):
        pass
    
    def prepare_targets(self, train_labels_df: pd.DataFrame) -> pd.DataFrame:
        """Prepare target information from pairs DataFrame.
        Args:
            pairs (pd.DataFrame): DataFrame containing 'pair' column.   
        Returns:
            pd.DataFrame: DataFrame with target information including price_1, price_2, and is_pair.
        """
        target_cols = [col for col in train_labels_df.columns if col not in ['timestamp', 'id']]
        display(target_cols)
        target_values = train_labels_df[target_cols]
        return target_values, target_cols
    
    def prepare_targets_info(self, pairs: pd.DataFrame) -> pd.DataFrame:
        """Prepare target information from pairs DataFrame.
        Args:
            pairs (pd.DataFrame): DataFrame containing 'pair' column.
        Returns:
            pd.DataFrame: DataFrame with target information including price_1, price_2, and is_pair.
        """

        target_definitions = pairs["pair"].str.split(" - ", expand=True)
        target_info = pairs.copy()

        # Colonnes price_1 et price_2 (équivalent aux colonnes [,1] et [,2])
        target_info["price_1"] = target_definitions[0]
        target_info["price_2"] = target_definitions[1]

        # is.pair = second élément non vide
        target_info['is_pair'] = target_info['price_2'].apply(lambda x:x is not None)

        # Retirer la colonne "pair"
        target_info = target_info.drop(columns=["pair"])
        return target_info

In [10]:
feature_engineer = FeatureEngineer()
train_features = feature_engineer.prepare_features(df_preprocess)
display(train_features.head())

,date,id,close,open,high,low,volume,sprice,interest,low_lag1,...,open_rollmin5,open_rollmax5,open_rollmean10,open_rollstd10,open_rollmin10,open_rollmax10,open_rollmean15,open_rollstd15,open_rollmin15,open_rollmax15
0,0,FX_AUDCAD,0.979601,23.9998,24.5242,23.89,2464147.0,4730.0,1768.0,57.258,...,23.9998,23.9998,23.9998,0.207041,23.9998,23.9998,23.9998,0.207041,23.9998,23.9998
2,0,FX_AUDJPY,87.933498,23.9998,24.5242,23.89,2464147.0,4730.0,1768.0,57.258,...,23.9998,23.9998,23.9998,0.207041,23.9998,23.9998,23.9998,0.207041,23.9998,23.9998
3,0,FX_AUDNZD,1.103011,23.9998,24.5242,23.89,2464147.0,4730.0,1768.0,57.258,...,23.9998,23.9998,23.9998,0.207041,23.9998,23.9998,23.9998,0.207041,23.9998,23.9998
4,0,FX_AUDUSD,0.783393,23.9998,24.5242,23.89,2464147.0,4730.0,1768.0,57.258,...,23.9998,23.9998,23.9998,0.207041,23.9998,23.9998,23.9998,0.207041,23.9998,23.9998
5,0,FX_CADCHF,0.776874,23.9998,24.5242,23.89,2464147.0,4730.0,1768.0,57.258,...,23.9998,23.9998,23.9998,0.207041,23.9998,23.9998,23.9998,0.207041,23.9998,23.9998


#  travail sur les cibles

In [ ]:
featureTarget = FeatureTarget()
target_values,target_cols = featureTarget.prepare_targets(train_labels_df)
target_info = featureTarget.prepare_targets_info(target_pairs_df)
# train target values
def convert_to_colname(colname:str)-> str:
    if colname is None:
        return None
    if colname.startswith("FX_"):
        return colname
    colname = colname.replace("_adj","")
    colname = colname.rsplit('_', 1)[0]
    return colname

def getnameprice(indexe_target:int):
    target_name = target_info.loc[indexe_target,'target']
    target_col1 = convert_to_colname(target_info.loc[indexe_target,'price_1'])
    target_col2 = convert_to_colname(target_info.loc[indexe_target,'price_2'])
    return  target_name, target_col1, target_col2

def train_features_build(target_col1,target_col2):
# Filter for the two targets
    if target_col2 is not None:
        df_target2 = train_features[train_features['id'] == target_col2]
        df_target1 = train_features[train_features['id'] == target_col1]
        
        # Concatenate the two DataFrames side by side (axis=1)
        result = df_target2.merge(
    df_target1,
    on='date',
    how='left',   # ou 'left'
    suffixes=('_t2', '_t1'))
        result = result.reset_index()
        result = result.drop(['id_t2', 'date_t2','id_t1', 'date_t1', 'index', 'date'], axis=1, errors='ignore')
        
        return result
    else:
    
        df_target1 = train_features[train_features['id'] == target_col1]
        # Concatenate the two DataFrames side by side (axis=1)
        result = df_target1
        result = result.reset_index()
        result = result.drop(['id', 'date', 'index'], axis=1, errors='ignore')
        
        return result

['target_0',
 'target_1',
 'target_2',
 'target_3',
 'target_4',
 'target_5',
 'target_6',
 'target_7',
 'target_8',
 'target_9',
 'target_10',
 'target_11',
 'target_12',
 'target_13',
 'target_14',
 'target_15',
 'target_16',
 'target_17',
 'target_18',
 'target_19',
 'target_20',
 'target_21',
 'target_22',
 'target_23',
 'target_24',
 'target_25',
 'target_26',
 'target_27',
 'target_28',
 'target_29',
 'target_30',
 'target_31',
 'target_32',
 'target_33',
 'target_34',
 'target_35',
 'target_36',
 'target_37',
 'target_38',
 'target_39',
 'target_40',
 'target_41',
 'target_42',
 'target_43',
 'target_44',
 'target_45',
 'target_46',
 'target_47',
 'target_48',
 'target_49',
 'target_50',
 'target_51',
 'target_52',
 'target_53',
 'target_54',
 'target_55',
 'target_56',
 'target_57',
 'target_58',
 'target_59',
 'target_60',
 'target_61',
 'target_62',
 'target_63',
 'target_64',
 'target_65',
 'target_66',
 'target_67',
 'target_68',
 'target_69',
 'target_70',
 'target_71',
 '

# Create train/test groups using DataLoader

In [16]:
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import torch.optim as optim
from sklearn.preprocessing import StandardScaler

class TimeSeriesDataset(Dataset):
    """Custom Dataset for time series data"""
    def __init__(self, X, y, sequence_length=10):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)
        self.sequence_length = sequence_length
        
    def __len__(self):
        return len(self.X) - self.sequence_length
        
    def __getitem__(self, idx):
        return (
            self.X[idx:idx + self.sequence_length],
            self.y[idx + self.sequence_length]
        )
    
def prepare_data(X, y, sequence_length=10, batch_size=32):
        """Prepare data for training"""
        # Scale features

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Create dataset and dataloader
        dataset = TimeSeriesDataset(X_scaled, y, sequence_length)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
        
        return dataloader, X_scaled

In [21]:
def train_models_for_target(
                p_target_values,
                target_col1:str,
                target_col2:str,
                batch_size:int=32
                ):
    
    y = p_target_values
    mask = ~y.isna()
    X_all = train_features_build(target_col1,target_col2)
    
    X_valid = X_all.loc[mask]
    y_valid = y[mask]

    # Simple train/validation split (last 20% for validation)
    split_idx = int(len(X_valid) * 0.8)
    X_train = X_valid[:split_idx]
    y_train = y_valid[:split_idx]
    X_val = X_valid[split_idx:]
    y_val = y_valid[split_idx:]
    
    display(X_train)
    display(y_train.values)
    # Prepare data
    train_loader, X_scaled = prepare_data(
                    X_train, y_train.values, 
                    10, batch_size
                )
    val_loader, _  = prepare_data(
                    X_val, y_val.values, 
                    10, batch_size
                )
    return train_loader, val_loader

In [37]:
#  Test du data loader
idx:int = 2
tgt=target_cols[idx]
y = target_values[tgt]
name,pric1,pric2 =getnameprice(idx)
train_loaders, val_loader = train_models_for_target(y,pric1,pric2)
display(train_loaders)
display(val_loader)


,close_t2,open_t2,high_t2,low_t2,volume_t2,sprice_t2,interest_t2,low_lag1_t2,low_lag2_t2,low_lag3_t2,...,open_rollmin5_t1,open_rollmax5_t1,open_rollmean10_t1,open_rollstd10_t1,open_rollmin10_t1,open_rollmax10_t1,open_rollmean15_t1,open_rollstd15_t1,open_rollmin15_t1,open_rollmax15_t1
0,3349.0,117.2076,119.9167,116.8076,1118561.0,4730.0,1768.0,57.2580,81.7687,57.2580,...,6.7563,6.7563,6.75630,0.207041,6.7563,6.7563,6.756300,0.207041,6.7563,6.7563
1,3327.0,26.1580,27.1420,26.1418,4954241.0,4730.0,1768.0,25.4966,81.7687,57.2580,...,6.7563,7.1514,6.95385,0.279378,6.7563,7.1514,6.953850,0.279378,6.7563,7.1514
2,3362.0,3363.0000,3443.0000,3362.0000,624.0,3423.0,1323.0,89.4475,89.3491,57.2580,...,57.3474,58.4289,57.80550,0.559378,57.3474,58.4289,57.805500,0.559378,57.3474,58.4289
3,3354.0,3426.0000,3494.0000,3425.0000,17629.0,3486.0,8800.0,3362.0000,47.1478,46.6155,...,3367.0000,3426.0000,3396.50000,41.719300,3367.0000,3426.0000,3396.500000,41.719300,3367.0000,3426.0000
4,3386.0,65.9713,65.9713,65.8740,2587313.0,204.8,8635.0,4727.0000,4679.0000,65.9389,...,4684.0000,4728.0000,4706.00000,31.112698,4684.0000,4728.0000,4706.000000,31.112698,4684.0000,4728.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,2314.5,4210.0000,4229.5000,4157.0000,482.0,4195.0,1003.0,4114.5000,90.1093,4101.0000,...,68.3649,69.9929,69.09848,0.546683,68.3649,69.9929,69.746467,1.077623,68.3649,71.5251
1458,2283.5,177.5164,181.9884,175.2268,1851231.0,4140.0,16379.0,174.6715,177.3995,181.9543,...,177.5164,190.5233,189.18262,9.617933,177.5164,209.0251,193.985887,10.499514,177.5164,209.0251
1459,2298.0,176.6201,179.9424,175.4120,2091277.0,196.3,11141.0,175.2268,174.6715,177.3995,...,5.8259,6.2581,5.97947,0.131207,5.8012,6.2581,6.088240,0.211821,5.8012,6.4667
1460,2300.0,176.1427,178.9486,175.6849,1775259.0,4171.0,16406.0,175.4120,175.2268,174.6715,...,176.1427,180.0301,183.98282,7.097356,176.1427,195.0050,190.141327,10.984448,176.1427,209.0251


array([-0.0046752 , -0.00705199, -0.00893406, ..., -0.00346242,
        0.00434168, -0.00593986], shape=(1396,))